In [2]:
from sympy import Symbol, simplify, Integer

In [2]:
import sympy as sp

# JIT compiler

In [18]:
x = sp.Symbol('x')

# Adding 2 to x
expr = x

In [23]:
expr-=2

In [20]:
expr.as_coeff_Add()

(2, x)

In [24]:
expr

x - 2

In [3]:

class SymbolicValue:
    def __init__(self, initial_value=None, symbolic=False):
        self.symbolic = symbolic
        if symbolic:
            self.value = Symbol('x')
        else:
            self.value = initial_value if initial_value is not None else 0

    def increment(self):
        self.value += 1
        if self.symbolic:
            self.value = simplify(self.value)

    def decrement(self):
        self.value -= 1
        if self.symbolic:
            self.value = simplify(self.value)

    def is_changed(self):
        # fix ze sa pozrie aj ci sa zmenila od posledna
        return self.value != 0 or self.symbolic
    
    def get_optimized_value(self):
        return simplify(self.value) if self.symbolic else self.value

    def reset(self):
        self.value = 0

    def __str__(self):
        return str(self.value)

    
class BrainfuckSymbolicSolver:
    def __init__(self):
        self.tape = [SymbolicValue() for _ in range(30000)]  
        self.pointer = 0
        self.symbolic_state = {}  # to track symbolic values
        self.loop_stack = [] # to keep track of loop
        
    def interpret(self, code):
        # sym interpret
        for char in code:
            self.execute_command(char)

    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char in ['+', '-']:
            self.symbolic_add_sub(char)
        # handel loops

    def symbolic_add_sub(self, char):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        if char == '+':
            self.symbolic_state[self.pointer].increment()
        elif char == '-':
            self.symbolic_state[self.pointer].decrement()


    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0
        self.symbolic_state = {}
        loop_started = False
        for char in code:
            if char in ['+', '-', '>', '<'] and not loop_started:
                self.execute_command(char)
            elif char in ['+', '-', '>', '<'] and loop_started:
                loop_content += char
            elif char == '[':
                loop_content = ""
                loop_started = True
            elif char == ']':
                loop_started = False
                self.handle_loop(loop_content)
            elif char == '.':
                optimized_code += self.apply_optimizations()
                optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"
            else:
                optimized_code += self.apply_optimizations()
                optimized_code += char

        # Apply any remaining optimizations at the end
        optimized_code += self.apply_optimizations()
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer, symbolic_value  in self.symbolic_state.items():
            if symbolic_value.is_changed():
                self.tape[pointer] = symbolic_value
                optimized_code += f"tape[{pointer}] = {symbolic_value.get_optimized_value()}\n"
                # value.reset()

        # self.symbolic_state = {}
        return optimized_code

    def handle_loop(self, code):
        current_i = self.symbolic_state[self.pointer]
    
    def optimize_and_convert_to_python(self, bf_filename):
        # with open(bf_filename, 'r') as file:
        #     bf_code = file.read()
        bf_code = bf_filename
        optimized_python_code = self.optimize(bf_code)
        # py_filename = bf_filename.rsplit('.', 1)[0] + '.py'
        py_filename = "test.py"
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))

        print(f"Python code generated: {py_filename}")
    
    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape
    pointer = 0
    
    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)
 


In [5]:
# solver = BrainfuckSymbolicSolver()
# solver.interpret("++>+-") 
# # solver.optimize()

In [6]:
# solver = BrainfuckSymbolicSolver()
# solver.optimize_and_convert_to_python("+.++--")

Python code generated: test.py


In [27]:
solver = BrainfuckSymbolicSolver()
optimized_code = solver.optimize("+++++>+++")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 5
tape[1] = 3



In [28]:
optimized_code = solver.optimize("++>+-")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 2
tape[1] = 0



# Partial Execution

In [4]:
class SymbolicValue:
    def __init__(self, initial_value=None, symbolic='x'):
    
        self.symbol = sp.Symbol(symbolic)
        
        if initial_value is None:
            self.expr = self.symbol
            self.is_changed = False
        else:
            self.expr = self.symbol + initial_value
            self.is_changed = True
        
    
    def get_concrete_val(self):
        concrete, _ = self.expr.as_coeff_Add()
        return int(concrete)

    def reset(self):
        self.expr = self.symbol

    def sym_add(self):
        self.expr += 1
        
    def __add__(self, n):
        # Creating a new instance with the updated expression
        # self.is_changed = True
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() + n)

    def __sub__(self, n):
        # self.is_changed = True
        # Creating a new instance with the updated expression
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() - n)

    def sym_sub(self):
        self.expr -= 1

    def __str__(self):
        # concrete, _ = self.expr.as_coeff_Add()
        return str(self.expr)

# Last ver

In [1]:
import copy
import os
import time
import sympy as sp

class SymbolicValue:
    def __init__(self, initial_value=None, symbolic='x'):

        self.symbol = sp.Symbol(symbolic)

        if initial_value is None:
            self.expr = self.symbol
            self.is_changed = False
        else:
            self.expr = self.symbol + initial_value
            self.is_changed = True

    def get_concrete_val(self):
        concrete, _ = self.expr.as_coeff_Add()
        return int(concrete)

    def reset(self):
        self.expr = self.symbol
    def copy(self):
        # Create a new instance with the same state
        new_copy = SymbolicValue(symbolic=str(self.symbol))
        new_copy.expr = self.expr
        new_copy.is_changed = self.is_changed
        return new_copy

    def __add__(self, n):
        # self.is_changed = True
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() + n)

    def __sub__(self, n):
        # Creating a new instance with the updated expression
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() - n)

    def __str__(self):
        # concrete, _ = self.expr.as_coeff_Add()
        return str(self.expr)


class BrainfuckSymbolicSolver:
    # LATEST
    def __init__(self):
        self.tape = [0 for _ in range(3000)]  # tape of size 3000
        self.pointer = 0
        self.loop_stack = []  # to keep track of loop
        self.last_state = [0 for _ in range(3000)]
        self.loop_map = {}
        self.dump = False

    def clear(self):
        self.tape = [0 for _ in range(3000)]
        self.pointer = 0
        self.loop_stack = []
        self.last_state = [0 for _ in range(3000)]
        self.loop_map = {}

    def handle_loop_start(self, index, loop_end):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] == 0:
                return loop_end  # Go to loop end
            else:
                self.loop_stack.append(self.pointer)
                return index
        elif isinstance(self.tape[self.pointer], SymbolicValue):
            self.dump = True
            return index-1
        return index

    def handle_loop_end(self, index, loop_start):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] != 0:
                #self.pointer = self.loop_stack[-1]
                return loop_start
            else:
                self.loop_stack.pop()
                return index
        elif isinstance(self.tape[self.pointer], SymbolicValue):
            self.dump = True
            return loop_start-1
        return index
    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char == '+':
            if self.tape[self.pointer] == 255:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] += 1
        elif char == '-':
            if self.tape[self.pointer] == 0:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] -= 1

    def execute_loops(self, char, index, code):
        if char == '[':
            loop_end = self.loop_map[index]
            return self.handle_loop_start(index, loop_end)
        elif char == ']':
            loop_start = self.loop_map[index]
            return self.handle_loop_end(index, loop_start)

    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0
        index = 0
        add_tab = 0
        while index < len(code):
            char = code[index]
            #print(f'Pointer = {self.pointer}, index = {index}, value = {self.tape[self.pointer]}, char = {char} ')
            if self.dump:
                if char == '>':
                    optimized_code += add_tab*"    "+"pointer += 1\n"
                
                elif char == '<':
                        optimized_code += add_tab*"    "+"pointer -= 1\n"
                    
                elif char == '+':
                    optimized_code += add_tab*"    "+"tape[pointer] = (tape[pointer] + 1)\n" 

                elif char == '-':
                    optimized_code += add_tab*"    "+"tape[pointer] = (tape[pointer] - 1)\n" 

                elif char == '.':
                    optimized_code += add_tab*"    "+"print(chr(tape[pointer]), end='')\n"

                elif char == ',':
                    optimized_code += add_tab*"    "+"tape[pointer] = ord(input('Input a character: ')[0])\n"

                elif char == '[':
                    add_tab += 1
                    optimized_code += "while tape[pointer] != 0:\n"

                elif char == ']':
                    add_tab-=1
                    optimized_code += "\n"
            else:
                if char in ['+', '-', '>', '<']:
                    self.execute_command(char)
                elif char == '[':
                    index = self.execute_loops(char, index, code)
                    if self.dump:
                        optimized_code += self.apply_optimizations()
                        optimized_code += f"pointer = {self.pointer}\n"
                elif char == ']':
                    index = self.execute_loops(char, index, code)
                    if self.dump:
                        optimized_code += self.apply_optimizations()
                        optimized_code += f"pointer = {self.pointer}\n"
                elif char == '.':
                    optimized_code += self.optimize_print()
                    # had_to_trans = True
                    optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"
                elif char == ',':
                    optimized_code += f"tape[{self.pointer}] = ord(input())\n"
                    self.tape[self.pointer] = SymbolicValue()

            # else:
            #     # If not in the supported op semantics, dont count index
            #     continue
            index += 1
        # if code[index - 1] != '.':
            # Apply any remaining optimizations at the end if it wasnt a print as last command
        if not self.dump:
            optimized_code += self.apply_optimizations()
        return optimized_code
    
    def optimize_print(self):
        optimized_code = ""
        if all(x == 0 for x in self.last_state):
            if isinstance(self.tape[self.pointer], SymbolicValue):
                if self.tape[self.pointer].get_concrete_val() != 0:
                    optimized_code += f"tape[{self.pointer}] += {self.tape[self.pointer].get_concrete_val()}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()

            else:
                optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                self.last_state[self.pointer] = self.tape[self.pointer]
            
        else:
            if isinstance(self.tape[self.pointer], SymbolicValue) and isinstance(self.last_state[self.pointer],SymbolicValue):
                # was and still is symbolic
                # if self.tape[self.pointer].get_concrete_val() != \
                #         self.last_state[self.pointer].get_concrete_val():
                if self.tape[self.pointer].is_changed:
                    diff = self.tape[self.pointer].get_concrete_val() - self.last_state[self.pointer].get_concrete_val()
                    if diff<0:
                        optimized_code += f"tape[{self.pointer}] -= {abs(diff)}\n"
                    elif diff!=0:
                        optimized_code += f"tape[{self.pointer}] += {diff}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()
                    self.tape[self.pointer].is_changed = False
            elif isinstance(self.tape[self.pointer], SymbolicValue):
                # current is symbolic - prev was concrete
                if self.tape[self.pointer].get_concrete_val() != 0:
                    optimized_code += f"tape[{self.pointer}] += {self.tape[self.pointer].get_concrete_val()}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()
            elif isinstance(self.last_state[self.pointer], SymbolicValue):
                # previous was symbolic - now is conrete
                optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                self.last_state[self.pointer] = self.tape[self.pointer]
            else:
                # always was concrete
                if (self.tape[self.pointer] != self.last_state[self.pointer]):
                    optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer]
            
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer in range(len(self.tape)):

            if all(x == 0 for x in self.last_state):
                if self.tape[pointer] == 0:
                    continue
                if isinstance(self.tape[pointer], SymbolicValue):
                    # Is a symbolic expression
                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                else:
                    # Is a concrete value
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
            else:
                if isinstance(self.tape[pointer], SymbolicValue) and isinstance(self.last_state[pointer],
                                                                                SymbolicValue):
                    # was and still is symbolic
                    if self.tape[pointer].is_changed:
                        diff = self.tape[pointer].get_concrete_val() - self.last_state[pointer].get_concrete_val()
                        if diff<0:
                            optimized_code += f"tape[{pointer}] -= {abs(diff)}\n"
                        elif diff!=0:
                            optimized_code += f"tape[{pointer}] += {diff}\n"
                        self.last_state[pointer] = self.tape[pointer].copy()
                        self.tape[pointer].is_changed = False
                elif isinstance(self.tape[pointer], SymbolicValue):
                    # current is symbolic - prev was concrete
                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.last_state[pointer], SymbolicValue):
                    # previous was symbolic - now is conrete
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                else:
                    # always was concrete
                    if (self.tape[pointer] != self.last_state[pointer]):
                        optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"

        return optimized_code

    def preprocess_loops(self, code):
        loop_map = {}
        loop_stack = []

        for i, char in enumerate(code):
            if char == '[':
                loop_stack.append(i)
            elif char == ']':
                if len(loop_stack) == 0:
                    raise ValueError(f"Unmatched ']' at position {i}")
                start_index = loop_stack.pop()
                loop_map[start_index] = i
                loop_map[i] = start_index

        if len(loop_stack) > 0:
            raise ValueError(f"Unmatched '[' at position {loop_stack[-1]}")

        return loop_map
    def optimize_and_convert_to_python(self, bf_filename, result_file_name = "test.py"):
        with open(bf_filename, 'r') as file:
             bf_code = file.read()

        self.loop_map = self.preprocess_loops(bf_code)
        optimized_python_code = self.optimize(bf_code)

        py_filename = os.path.join("out",result_file_name)
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))
        print(f"Python code generated: {py_filename}")

    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape

    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)

In [4]:
solver = BrainfuckSymbolicSolver()
solver.optimize_and_convert_to_python("C:\\Users\\peter\\Desktop\\CS msc\\1st semester\\Program Analysis\\bfTest]\\Program_analysis_brainfuck\\Bf_test_code\\triangle.b", result_file_name='test-m2.py')

Python code generated: out\test-m2.py


# Old

In [25]:
class SymbolicValue:
    def __init__(self, initial_value=None, symbolic='x'):

        self.symbol = sp.Symbol(symbolic)

        if initial_value is None:
            self.expr = self.symbol
            self.is_changed = False
        else:
            self.expr = self.symbol + initial_value
            self.is_changed = True

    def get_concrete_val(self):
        concrete, _ = self.expr.as_coeff_Add()
        return int(concrete)

    def reset(self):
        self.expr = self.symbol

    def sym_add(self):
        self.expr += 1

    def __add__(self, n):
        # Creating a new instance with the updated expression
        # self.is_changed = True
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() + n)

    def __sub__(self, n):
        # self.is_changed = True
        # Creating a new instance with the updated expression
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() - n)

    def sym_sub(self):
        self.expr -= 1

    def __str__(self):
        # concrete, _ = self.expr.as_coeff_Add()
        return str(self.expr)


class BrainfuckSymbolicSolver_old:
    # LATEST
    def __init__(self):
        self.tape = [0 for _ in range(3000)]  # tape of size 3000
        self.pointer = 0
        self.symbolic_state = {}  # to track the state - mixed sym and concrete
        self.loop_stack = []  # to keep track of loop
        self.last_state = []
        self.loop_map = {}

    def interpret(self, code):
        #  interpreter
        for char in code:
            self.execute_command(char)

    def get_loop_start(self, index, code):
        counter = 1  # counter to handle nested loops
        for i in range(index - 1, -1, -1):
            if code[i] == ']':
                counter += 1
            elif code[i] == '[':
                counter -= 1
                if counter == 0:
                    return i
        return -1

    def get_loop_end(self, index, code):
        counter = 1  # counter to handle nested loops
        for i in range(index + 1, len(code)):
            if code[i] == '[':
                counter += 1
            elif code[i] == ']':
                counter -= 1
                if counter == 0:
                    return i
        return -1

    def handle_loop_start(self, index, loop_end):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] == 0:
                return loop_end  # Go to loop end
            else:
                self.loop_stack.append(self.pointer)
                return index
        return index

    def handle_loop_end(self, index, loop_start):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] != 0:
                #self.pointer = self.loop_stack[-1]
                return loop_start
            else:
                self.loop_stack.pop()
                return index
        return index
    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char == '+':
            if self.tape[self.pointer] == 255:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] += 1
        elif char == '-':
            if self.tape[self.pointer] == 0:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] -= 1
        # handel loops

    def execute_loops(self, char, index, code):
        if char == '[':
            loop_end = self.loop_map[index]
            return self.handle_loop_start(index, loop_end)
        elif char == ']':
            loop_start = self.loop_map[index]
            return self.handle_loop_end(index, loop_start)

    def symbolic_add_sub(self, char):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        if char == '+':
            self.symbolic_state[self.pointer].increment()
        elif char == '-':
            self.symbolic_state[self.pointer].decrement()

    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0

        # had_to_trans = False
        index = 0
        while index < len(code):
            char = code[index]
            #print(f'Pointer = {self.pointer}, index = {index}, value = {self.tape[self.pointer]}, char = {char} ')
            if char in ['+', '-', '>', '<']:
                self.execute_command(char)
            elif char == '[':
                index = self.execute_loops(char, index, code)
            elif char == ']':
                index = self.execute_loops(char, index, code)
            elif char == '.':
                optimized_code += self.apply_optimizations()
                # had_to_trans = True
                optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"

            elif char == ',':
                optimized_code += f"tape[{self.pointer}] = input()\n"
                self.tape[self.pointer] = SymbolicValue()

            # else:
            # optimized_code += self.apply_optimizations()
            # optimized_code += char
            index += 1
        if code[index - 1] != '.':
            # Apply any remaining optimizations at the end if it wasnt a print as last command
            optimized_code += self.apply_optimizations()
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer in range(len(self.tape)):

            if self.last_state == []:
                if self.tape[pointer] == 0:
                    continue
                if isinstance(self.tape[pointer], SymbolicValue):
                    # Is a symbolic expression
                    # optimized_code += f"tape[{pointer}] = input()\n"

                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                    # else:
                    #     optimized_code += f"tape[{pointer}] = input()\n"
                else:
                    # Is a concrete value
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                    # value.reset()
            else:
                if isinstance(self.tape[pointer], SymbolicValue) and isinstance(self.last_state[pointer],
                                                                                SymbolicValue):
                    # was and still is symbolic
                    if self.tape[pointer].get_concrete_val() != 0 and self.tape[pointer].get_concrete_val() != \
                            self.last_state[pointer].get_concrete_val():
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.tape[pointer], SymbolicValue):
                    # current is symbolic - prev was concrete
                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.last_state[pointer], SymbolicValue):
                    # previous was symbolic - now is conrete
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                else:
                    # always was concrete
                    if (self.tape[pointer] != self.last_state[pointer]):
                        optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"

        self.last_state = copy.deepcopy(self.tape)
        return optimized_code

    def handle_loop(self, code):
        current_i = self.symbolic_state[self.pointer]

    def preprocess_loops(self, code):
        loop_map = {}
        loop_stack = []

        for i, char in enumerate(code):
            if char == '[':
                loop_stack.append(i)
            elif char == ']':
                if len(loop_stack) == 0:
                    raise ValueError(f"Unmatched ']' at position {i}")
                start_index = loop_stack.pop()
                loop_map[start_index] = i
                loop_map[i] = start_index

        if len(loop_stack) > 0:
            raise ValueError(f"Unmatched '[' at position {loop_stack[-1]}")

        return loop_map
    def optimize_and_convert_to_python(self, bf_filename, result_file_name = "test.py"):
        with open(bf_filename, 'r') as file:
             bf_code = file.read()
        #bf_code = bf_filename
        bf_code_striped = bf_code.strip()
        bf_code_no_new_line = bf_code.replace("\n", "")
        print(bf_code_no_new_line)
        self.loop_map = self.preprocess_loops(bf_code_no_new_line)
        optimized_python_code = self.optimize(bf_code_no_new_line)
        # py_filename = bf_filename.rsplit('.', 1)[0] + '.py'
        py_filename = result_file_name
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))

        print(f"Python code generated: {py_filename}")

    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape
    pointer = 0

    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)


In [27]:
solver = BrainfuckSymbolicSolver_old()
solver.optimize_and_convert_to_python("C:\\Users\\peter\\Desktop\\CS msc\\1st semester\\Program Analysis\\bfTest]\\Program_analysis_brainfuck\\Bf_test_code\\beer.b", result_file_name='test-old.py')

>++++++++++[<++++++++++>-]<->>>>>+++[>+++>+++<<-]<<<<+<[>[>+>+<<-]>>[-<<+>>]++++>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<[[-]>>>>>>[[-]<++++++++++<->>]<-[>+>+<<-]>[<+>-]+>[[-]<->]<<<<<<<<<->>]<[>+>+<<-]>>[-<<+>>]+>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<<[>>+>+<<<-]>>>[-<<<+>>>]++>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<[>+<[-]]<[>>+<<[-]]>>[<<+>>[-]]<<<[>>+>+<<<-]>>>[-<<<+>>>]++++>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<[>+<[-]]<[>>+<<[-]]>>[<<+>>[-]]<<[[-]>>>++++++++[>>++++++<<-]>[<++++++++[>++++++<-]>.<++++++++[>------<-]>[<<+>>-]]>.<<++++++++[>>------<<-]<[->>+<<]<++++++++[<++++>-]<.>+++++++[>+++++++++<-]>+++.<+++++[>+++++++++<-]>.+++++..--------.-------.++++++++++++++>>[>>>+>+<<<<-]>>>>[-<<<<+>>>>]>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<<<[>>>+>+<<<<-]>>>>[-<<<<+>>>>]+>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<<[>>+<<[-]]>[>+<[-]]++>>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<+<[[-]>-<]>[<<<<<<<.>>>>>>>[-]]<<<<<<<<<.>>----.---------.<<.>>----.+++..+++++++++++++.[-]<<[-]]<[>+>+<<-]>>[-<<+>>]+>+<[-<->]<[[-]>>-<<]>>[[-]<<+>>]<<<[>>+>+<<

In [ ]:
import copy
import os
import time
import sympy as sp

class SymbolicValue:
    def __init__(self, initial_value=None, symbolic='x'):

        self.symbol = sp.Symbol(symbolic)

        if initial_value is None:
            self.expr = self.symbol
            self.is_changed = False
        else:
            self.expr = self.symbol + initial_value
            self.is_changed = True

    def get_concrete_val(self):
        concrete, _ = self.expr.as_coeff_Add()
        return int(concrete)

    def reset(self):
        self.expr = self.symbol

    def sym_add(self):
        self.expr += 1

    def __add__(self, n):
        # Creating a new instance with the updated expression
        # self.is_changed = True
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() + n)

    def __sub__(self, n):
        # self.is_changed = True
        # Creating a new instance with the updated expression
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() - n)

    def sym_sub(self):
        self.expr -= 1

    def __str__(self):
        # concrete, _ = self.expr.as_coeff_Add()
        return str(self.expr)


class BrainfuckSymbolicSolver:
    # LATEST
    def __init__(self):
        self.tape = [0 for _ in range(3000)]  # tape of size 3000
        self.pointer = 0
        self.symbolic_state = {}  # to track the state - mixed sym and concrete
        self.loop_stack = []  # to keep track of loop
        self.last_state = []
        self.loop_map = {}

    def clear(self):
        self.tape = [0 for _ in range(3000)]
        self.pointer = 0
        self.symbolic_state = {}
        self.loop_stack = []
        self.last_state = []
        self.loop_map = {}

    def handle_loop_start(self, index, loop_end):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] == 0:
                return loop_end  # Go to loop end
            else:
                self.loop_stack.append(self.pointer)
                return index
        return index

    def handle_loop_end(self, index, loop_start):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] != 0:
                #self.pointer = self.loop_stack[-1]
                return loop_start
            else:
                self.loop_stack.pop()
                return index
        return index
    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char == '+':
            if self.tape[self.pointer] == 255:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] += 1
        elif char == '-':
            if self.tape[self.pointer] == 0:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] -= 1
        # handel loops

    def execute_loops(self, char, index, code):
        if char == '[':
            loop_end = self.loop_map[index]
            return self.handle_loop_start(index, loop_end)
        elif char == ']':
            loop_start = self.loop_map[index]
            return self.handle_loop_end(index, loop_start)

    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0

        # had_to_trans = False
        index = 0
        while index < len(code):
            char = code[index]
            #print(f'Pointer = {self.pointer}, index = {index}, value = {self.tape[self.pointer]}, char = {char} ')
            if char in ['+', '-', '>', '<']:
                self.execute_command(char)
            elif char == '[':
                index = self.execute_loops(char, index, code)
            elif char == ']':
                index = self.execute_loops(char, index, code)
            elif char == '.':
                optimized_code += self.optimize_print()
                # had_to_trans = True
                optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"

            elif char == ',':
                optimized_code += f"tape[{self.pointer}] = input()\n"
                self.tape[self.pointer] = SymbolicValue()

            # else:
            # optimized_code += self.apply_optimizations()
            # optimized_code += char
            index += 1
        if code[index - 1] != '.':
            # Apply any remaining optimizations at the end if it wasnt a print as last command
            optimized_code += self.apply_optimizations()
        return optimized_code
    
    def optimize_print(self):
        optimized_code = ""
        if isinstance(self.tape[self.pointer], SymbolicValue):
            if self.tape[self.pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{self.pointer}] += {self.tape[self.pointer].get_concrete_val()}\n"
        else:
            optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer in range(len(self.tape)):

            if self.last_state == []:
                if self.tape[pointer] == 0:
                    continue
                if isinstance(self.tape[pointer], SymbolicValue):
                    # Is a symbolic expression
                    # optimized_code += f"tape[{pointer}] = input()\n"

                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                    # else:
                    #     optimized_code += f"tape[{pointer}] = input()\n"
                else:
                    # Is a concrete value
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                    # value.reset()
            else:
                if isinstance(self.tape[pointer], SymbolicValue) and isinstance(self.last_state[pointer],
                                                                                SymbolicValue):
                    # was and still is symbolic
                    if self.tape[pointer].get_concrete_val() != 0 and self.tape[pointer].get_concrete_val() != \
                            self.last_state[pointer].get_concrete_val():
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.tape[pointer], SymbolicValue):
                    # current is symbolic - prev was concrete
                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.last_state[pointer], SymbolicValue):
                    # previous was symbolic - now is conrete
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                else:
                    # always was concrete
                    if (self.tape[pointer] != self.last_state[pointer]):
                        optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"

        self.last_state = copy.deepcopy(self.tape)
        return optimized_code

    def preprocess_loops(self, code):
        loop_map = {}
        loop_stack = []

        for i, char in enumerate(code):
            if char == '[':
                loop_stack.append(i)
            elif char == ']':
                if len(loop_stack) == 0:
                    raise ValueError(f"Unmatched ']' at position {i}")
                start_index = loop_stack.pop()
                loop_map[start_index] = i
                loop_map[i] = start_index

        if len(loop_stack) > 0:
            raise ValueError(f"Unmatched '[' at position {loop_stack[-1]}")

        return loop_map
    def optimize_and_convert_to_python(self, bf_filename, result_file_name = "test.py"):
        with open(bf_filename, 'r') as file:
             bf_code = file.read()

        self.loop_map = self.preprocess_loops(bf_code)
        optimized_python_code = self.optimize(bf_code)


In [20]:
import copy
import os
import time
import sympy as sp

class SymbolicValue:
    def __init__(self, initial_value=None, symbolic='x'):

        self.symbol = sp.Symbol(symbolic)

        if initial_value is None:
            self.expr = self.symbol
            self.is_changed = False
        else:
            self.expr = self.symbol + initial_value
            self.is_changed = True

    def get_concrete_val(self):
        concrete, _ = self.expr.as_coeff_Add()
        return int(concrete)

    def reset(self):
        self.expr = self.symbol

    def sym_add(self):
        self.expr += 1

    def __add__(self, n):
        # Creating a new instance with the updated expression
        # self.is_changed = True
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() + n)

    def __sub__(self, n):
        # self.is_changed = True
        # Creating a new instance with the updated expression
        return SymbolicValue(symbolic=str(self.symbol), initial_value=self.get_concrete_val() - n)

    def sym_sub(self):
        self.expr -= 1

    def copy(self):
        # Create a new instance with the same state
        new_copy = SymbolicValue(symbolic=str(self.symbol))
        new_copy.expr = self.expr
        new_copy.is_changed = self.is_changed
        return new_copy

    def __str__(self):
        # concrete, _ = self.expr.as_coeff_Add()
        return str(self.expr)


class BrainfuckSymbolicSolver:
    # LATEST
    def __init__(self):
        self.tape = [0 for _ in range(3000)]  # tape of size 3000
        self.pointer = 0
        # self.symbolic_state = {}  # to track the state - mixed sym and concrete
        self.loop_stack = []  # to keep track of loop
        self.last_state = [0 for _ in range(3000)]
        self.loop_map = {}

    def interpret(self, code):
        #  interpreter
        for char in code:
            self.execute_command(char)

    def get_loop_start(self, index, code):
        counter = 1  # counter to handle nested loops
        for i in range(index - 1, -1, -1):
            if code[i] == ']':
                counter += 1
            elif code[i] == '[':
                counter -= 1
                if counter == 0:
                    return i
        return -1

    def get_loop_end(self, index, code):
        counter = 1  # counter to handle nested loops
        for i in range(index + 1, len(code)):
            if code[i] == '[':
                counter += 1
            elif code[i] == ']':
                counter -= 1
                if counter == 0:
                    return i
        return -1

    def handle_loop_start(self, index, loop_end):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] == 0:
                return loop_end  # Go to loop end
            else:
                self.loop_stack.append(self.pointer)
                return index
        return index

    def handle_loop_end(self, index, loop_start):
        if isinstance(self.tape[self.pointer], int):
            if self.tape[self.pointer] != 0:
                #self.pointer = self.loop_stack[-1]
                return loop_start
            else:
                self.loop_stack.pop()
                return index
        return index
    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char == '+':
            if self.tape[self.pointer] == 255:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] += 1
        elif char == '-':
            if self.tape[self.pointer] == 0:
                self.tape[self.pointer] = 0
            else:
                self.tape[self.pointer] -= 1
        # handel loops

    def execute_loops(self, char, index, code):
        if char == '[':
            loop_end = self.loop_map[index]
            return self.handle_loop_start(index, loop_end)
        elif char == ']':
            loop_start = self.loop_map[index]
            return self.handle_loop_end(index, loop_start)

    def symbolic_add_sub(self, char):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        if char == '+':
            self.symbolic_state[self.pointer].increment()
        elif char == '-':
            self.symbolic_state[self.pointer].decrement()

    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0

        # had_to_trans = False
        index = 0
        while index < len(code):
            char = code[index]
            #print(f'Pointer = {self.pointer}, index = {index}, value = {self.tape[self.pointer]}, char = {char} ')
            if char in ['+', '-', '>', '<']:
                self.execute_command(char)
            elif char == '[':
                index = self.execute_loops(char, index, code)
            elif char == ']':
                index = self.execute_loops(char, index, code)
            elif char == '.':
                optimized_code += self.optimize_print()
                # had_to_trans = True
                optimized_code += f"print(chr(tape[{self.pointer}]), end='')\n"

            elif char == ',':
                optimized_code += f"tape[{self.pointer}] = input()\n"
                self.tape[self.pointer] = SymbolicValue()

            # else:
            # optimized_code += self.apply_optimizations()
            # optimized_code += char
            index += 1
        # if code[index - 1] != '.':
            # Apply any remaining optimizations at the end if it wasnt a print as last command
        optimized_code += self.apply_optimizations()
        return optimized_code
    
    def optimize_print(self):
        optimized_code = ""
        if all(x == 0 for x in self.last_state):
            if isinstance(self.tape[self.pointer], SymbolicValue):
                if self.tape[self.pointer].get_concrete_val() != 0:
                    optimized_code += f"tape[{self.pointer}] += {self.tape[self.pointer].get_concrete_val()}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()

            else:
                optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                self.last_state[self.pointer] = self.tape[self.pointer]
            
        else:
            if isinstance(self.tape[self.pointer], SymbolicValue) and isinstance(self.last_state[self.pointer],SymbolicValue):
                # was and still is symbolic
                if self.tape[self.pointer].get_concrete_val() != \
                        self.last_state[self.pointer].get_concrete_val():
                    diff = self.tape[self.pointer].get_concrete_val() - self.last_state[self.pointer].get_concrete_val()
                    if diff<0:
                        optimized_code += f"tape[{self.pointer}] -= {abs(diff)}\n"
                    elif diff!=0:
                        optimized_code += f"tape[{self.pointer}] += {diff}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()
            elif isinstance(self.tape[self.pointer], SymbolicValue):
                # current is symbolic - prev was concrete
                if self.tape[self.pointer].get_concrete_val() != 0:
                    optimized_code += f"tape[{self.pointer}] += {self.tape[self.pointer].get_concrete_val()}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer].copy()
            elif isinstance(self.last_state[self.pointer], SymbolicValue):
                # previous was symbolic - now is conrete
                optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                self.last_state[self.pointer] = self.tape[self.pointer]
            else:
                # always was concrete
                if (self.tape[self.pointer] != self.last_state[self.pointer]):
                    optimized_code += f"tape[{self.pointer}] = {self.tape[self.pointer]}\n"
                    self.last_state[self.pointer] = self.tape[self.pointer]
            
        return optimized_code

    def apply_optimizations(self):
        optimized_code = ""
        for pointer in range(len(self.tape)):

            if self.last_state == []:
                if self.tape[pointer] == 0:
                    continue
                if isinstance(self.tape[pointer], SymbolicValue):
                    # Is a symbolic expression
                    # optimized_code += f"tape[{pointer}] = input()\n"

                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                    # else:
                    #     optimized_code += f"tape[{pointer}] = input()\n"
                else:
                    # Is a concrete value
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                    # value.reset()
            else:
                if isinstance(self.tape[pointer], SymbolicValue) and isinstance(self.last_state[pointer],
                                                                                SymbolicValue):
                    # was and still is symbolic
                    if self.tape[pointer].get_concrete_val() != 0 and self.tape[pointer].get_concrete_val() != \
                            self.last_state[pointer].get_concrete_val():
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.tape[pointer], SymbolicValue):
                    # current is symbolic - prev was concrete
                    if self.tape[pointer].get_concrete_val() != 0:
                        optimized_code += f"tape[{pointer}] += {self.tape[pointer].get_concrete_val()}\n"
                elif isinstance(self.last_state[pointer], SymbolicValue):
                    # previous was symbolic - now is conrete
                    optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"
                else:
                    # always was concrete
                    if (self.tape[pointer] != self.last_state[pointer]):
                        optimized_code += f"tape[{pointer}] = {self.tape[pointer]}\n"

        self.last_state = copy.deepcopy(self.tape)
        return optimized_code

    # def handle_loop(self, code):
    #     current_i = self.symbolic_state[self.pointer]

    def preprocess_loops(self, code):
        loop_map = {}
        loop_stack = []

        for i, char in enumerate(code):
            if char == '[':
                loop_stack.append(i)
            elif char == ']':
                if len(loop_stack) == 0:
                    raise ValueError(f"Unmatched ']' at position {i}")
                start_index = loop_stack.pop()
                loop_map[start_index] = i
                loop_map[i] = start_index

        if len(loop_stack) > 0:
            raise ValueError(f"Unmatched '[' at position {loop_stack[-1]}")

        return loop_map
    def optimize_and_convert_to_python(self, bf_filename, result_file_name = "test.py"):
        with open(bf_filename, 'r') as file:
             bf_code = file.read()
        #bf_code = bf_filename
        bf_code_striped = bf_code.strip()
        bf_code_no_new_line = bf_code.replace("\n", "")
        # print(bf_code_no_new_line)
        self.loop_map = self.preprocess_loops(bf_code_no_new_line)
        optimized_python_code = self.optimize(bf_code_no_new_line)
        # py_filename = bf_filename.rsplit('.', 1)[0] + '.py'
        py_filename = os.path.join("out",result_file_name)
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))
        print(f"Python code generated: {py_filename}")

    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape

    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)

In [21]:
solver = BrainfuckSymbolicSolver()
solver.optimize_and_convert_to_python("C:\\Users\\peter\\Desktop\\CS msc\\1st semester\\Program Analysis\\bfTest]\\Program_analysis_brainfuck\\Bf_test_code\\test.b", result_file_name='test.py')

Python code generated: out\test.py


In [7]:
with open("mandelbrot.b", 'r') as file:
            bf_code = file.read()

In [14]:
for i, char in enumerate(bf_code):
    if i == 153:
        print(char)

[
